# Energy Plots
for individual layers

In [1]:
%run '00_Imports-and-settings.ipynb'

Numpy Version:		 1.19.0
Pandas Version:		 1.0.5
Matplotlib Version:	 3.2.2


In [2]:
# helper function to get model name
def get_model_name(model_string):
    if 'ResNet' in model_string:
        model_name = 'ResNet20_CIFAR-10'
    elif 'LeNet' in model_string:
        model_name = 'LeNet-MNIST'
    else:
        model_name = 'unknown'
        print('unknown model name')
    return model_name

## Energy per layer

In [3]:
df = pd.read_pickle('results_energy/A_Aggregated_energy_inference_layer_table.pkl')

In [4]:
df['model_type'] = df['model_type'].str.replace('_','-')
df['MCU'] = df['MCU'].str.replace('_','-')

In [5]:
di = {"NUCLEO-L496ZG": "L4", "DISCO-F469NI": "F4", "NUCLEO-F767ZI": "F7"}

df = df.replace({"MCU": di})

In [6]:
df.columns

Index(['time', 'filename', 'MCU', 'model', 'model_name', 'mbed-dir',
       'cmsis-nn', 'compiler_optimization', 'FPU_status', 'model_type',
       'weights', 'activations', 'pruned', 'no_correct_inferences', 'layer',
       'layer_latency_mean', 'layer_latency_std', 'layer_sample_length_mean',
       'layer_sample_length_std', 'layer_voltage_mean', 'layer_voltage_std',
       'layer_current_mean', 'layer_current_std', 'layer_power_mean',
       'layer_power_std', 'layer_energy_mean', 'layer_energy_std',
       'layer_latency_relativ', 'layer_energy_relativ'],
      dtype='object')

## Linear Regression across all layers - absolute

In [7]:
df_filtered = df.copy()

# let's sort the filtered df so that the order in the plots match
df_filtered = df_filtered.sort_values(by=['model_type','MCU'])


df_filtered.loc[:,('layer_energy_mean')] *= 1000


fig = sns.lmplot(x="layer_latency_mean", y="layer_energy_mean", hue="MCU", data=df_filtered, height=4, aspect=16/9, hue_order=["L4", "F4", "F7"])



#plt.title('Regression accross all model, all MCUs, all layers')

plt.xlabel("Latency of the Layer [s]")
plt.ylabel("Layer Energy Consumption [mJ]")


filename = f'figures/layers/energy/regression_absolute'

print(filename)
plt.legend(['L4', 'F4', 'F7'])
plt.tight_layout()
plt.show()
plt.savefig(filename + '.pdf')
tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

figures/layers/energy/regression_absolute


## Linear Regression across all layers - relative

In [8]:
df_filtered = df.copy()

# let's sort the filtered df so that the order in the plots match
df_filtered = df_filtered.sort_values(by=['model_type'])


df_filtered.loc[:,('layer_energy_mean')] *= 1000
df_filtered = df_filtered.sort_values(by=['model_type','MCU'])


fig = sns.lmplot(x="layer_latency_relativ", y="layer_energy_relativ", hue="MCU", data=df_filtered, height=4, aspect=16/9, hue_order=["L4", "F4", "F7"])



#plt.title('Regression accross all model, all MCUs, all layers')

plt.xlabel("Layer Inference Latency Relative")
plt.ylabel("Layer Energy Consumption Relative")


filename = f'figures/layers/energy/regression_relative'

print(filename)
plt.legend()
plt.tight_layout()
plt.show()
plt.savefig(filename + '.pdf')
tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

figures/layers/energy/regression_relative


## Average Power across all layers

In [9]:
# model_type = 'W-int8-A-int8'
# model_type = 'W-float32-A-float32'

hue = 'MCU'
models = df['model_name'].unique()
print(models)

filter_dict = {
    'cmsis-nn': 'cmsis-nn',
    #'model_type': 'W-int8-A-int8',
    'model_type': 'W-int8-A-int8'
}


layers = [0,1,2,3,4,14,15,16,27,28,39,30,31,32,33,34,35]
# '0_QUANTIZE',
# '1_CONV_2D',
# '2_CONV_2D',
# '3_CONV_2D',
# '4_ADD',
# '14_ADD',
# '15_CONV_2D',
# '16_CONV_2D',
# '27_ADD',
# '28_CONV_2D',
# '29_CONV_2D',
# '30_ADD',
# '31_AVERAGE_POOL_2D',
# '32_RESHAPE',
# '33_FULLY_CONNECTED ',
# '34_SOFTMAX',
# '35_DEQUANTIZE',
# ]

for model in models:

    model_name = get_model_name(model)
    
    filter = (df['model_name'] == model)
    filter_descr = ''
    for key in filter_dict:
        filter = filter & (df[key] == filter_dict[key])
        filter_descr += (str(filter_dict[key]) + '_')

        
    if 'ResNet' in model:
        filter = filter * (df['layer'].isin(layers))


    df_filtered = df.loc[filter]

    # let's sort the filtered df so that the order in the plots match
    #model_type = df_filtered['model_type'].unique()[0]

    df_filtered = df_filtered.sort_values(by=['model_type', 'MCU'])
    df_filtered.loc[:,('layer')] = df_filtered.loc[:,('layer')].astype('category')
    df_filtered.loc[:,('layer_power_mean')] *= 1000

    df_filtered = df_filtered.sort_values(by=['model_type','MCU'])



    fig = sns.catplot(y='layer', x='layer_power_mean', kind='bar', hue=hue, data=df_filtered, legend=False, hue_order=["L4", "F4", "F7"])
#     display(df_filtered.Code.map(df_filtered.groupby(['MCU']).Value.mean()))
#     display(df_filtered.groupby('MCU').transform('mean'))
    display(df_filtered.groupby('MCU')['layer_power_mean'].mean())
    display(df_filtered.groupby('MCU')['layer_power_mean'].std())

    #plt.title( model_name + '-' + filter_dict['model_type'] + '_' +'Power')

    plt.xlabel("Power [mW]")
    if "ResNet" in model:
        plt.ylabel("Selected Layers")
    else:
        plt.ylabel("Layers")
    plt.legend(loc='best')

#     plt.legend(['L4', 'F4', 'F7'],loc='best')
    filename = f"figures/layers/energy/{model_name}_{filter_dict['model_type']}_power_hue-{hue}"
    



    print(filename)
    plt.tight_layout()
    plt.show()
    plt.savefig(filename + '.pdf')
    tikzplotlib.save(filename + '.tex', axis_width=AXIS_WIDTH, axis_height=AXIS_HEIGHT)

['01d_ResNet20_CIFAR-10' 'LeNet-MNIST']


/usr/local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:177: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

MCU
F4    231.29668225
F7    406.56370319
L4     43.91426761
Name: layer_power_mean, dtype: float64

MCU
F4     6.93840755
F7    25.76138725
L4     1.13501853
Name: layer_power_mean, dtype: float64

figures/layers/energy/ResNet20_CIFAR-10_W-int8-A-int8_power_hue-MCU


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

MCU
F4    232.19875439
F7    383.31982653
L4     43.00935471
Name: layer_power_mean, dtype: float64

MCU
F4    10.01258471
F7    22.91306287
L4     0.98488045
Name: layer_power_mean, dtype: float64

figures/layers/energy/LeNet-MNIST_W-int8-A-int8_power_hue-MCU
